In [13]:
import funct
import numpy as np
import pandas as pd
import os
import importlib
import IPython
import netCDF4 as nc
funct = importlib.reload(funct)

In [33]:
freq = 3

# Get the current working directory
current_dir = "/".join(
        IPython.extract_module_locals()[1]["__vsc_ipynb_file__"].split("/")[-5:]
    )
# Get the parent directory path
parent_dir = os.path.dirname(os.path.dirname(current_dir))

# Change the directory (optional)
os.chdir(parent_dir)
# Now, your current working directory is the parent folder.
rela_path_base = "Persian_CCS_201901_202403\CCS_esmJSON_2024-03-08075032am_"
df_list = []
for month in funct.iterate_months(2019, 1, 2024, 3):
    rela_path = rela_path_base + month + ".nc"
    file_path = os.path.join(parent_dir,rela_path)
    df = funct.create_time_series_dataframe(freq, rela_path)
    df_list.append(df)
df = pd.concat(df_list)
# orginal shape of the Esmeraldas basin is of size 53*52
# New flat array of 2756 pixels. Matches the original dimensions
df = df.replace(-99,np.nan)

# Drafts Below

In [19]:


# Create a sample DataFrame (replace with your data)
df = df[0:100]
# Lists to store non-NaN values and indices
non_nan_values = []
non_nan_indices = []

# Iterate through each row (axis=0)
for index, row in df.iterrows():
  # Iterate through each value in the row
  for col, value in row.items():
    if not pd.isna(value) and value !=0 :  # Check for non-NaN using isna
      non_nan_values.append(value)
      non_nan_indices.append((index, col))  # Store index and column name as a tuple

# Print the results
print("Non-NaN Values:", non_nan_values)
print("Non-NaN Indices:", non_nan_indices)


Non-NaN Values: [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.0, 1.0, 1.0, 3.0, 1.0, 1.0, 3.0, 28.0, 1.0, 1.0, 1.0, 4.0, 7.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 9.0, 3.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 3.0, 14.0, 14.0, 10.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 3.0, 9.0, 5.0, 9.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 5.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 3.0, 3.0, 4.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 2.0, 4.0, 15.0, 10.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 4.0, 11

In [53]:
df_array = df.loc['2023-01-20 00:00:00']
df_array = np.array(df_array)

In [54]:
file_path = 'Scripts/clipped_ds_3h/20230120HH00__20230120HH03.nc'

# Open the file in read-only mode
dataset = nc.Dataset(file_path, mode='r')
dataset = dataset['precipitation']
array = np.array(dataset[0].data)

In [55]:
a = df_array.reshape(53,52)
b = array